# 2026-02-14.ipynb

- Reference: https://aubreymoore.github.io/jb/create-one-to-many-sql/

## TODO

- [] move `delete_results_from_gpu_memory()` function to `roadside.py`
- [] move `populate_detections_table` function to `roadside.py`
- [x] remove detection attribute fields from schema
- [] populate fields in `images` table
- [] prevent duplication of records in database

In [1]:
import roadside as rs
import sqlite3
import os
from icecream import ic

# Global variables for this run

In [2]:
root_dir = "/home/aubrey/Desktop/sam3-2026-01-31"
image_paths = ["20251129_152106.jpg", "08hs-palms-03-zglw-superJumbo.webp" ]
text_prompts=["coconut palm tree"]
db_path = 'sam3_detections.sqlite3'
schema_sql = """
--- 2026-02-14 07:32

CREATE TABLE IF NOT EXISTS images (
  image_id INTEGER PRIMARY KEY,
  image_path TEXT UNIQUE,
  image_width INTEGER,
  image_height INTEGER,
  timestamp TEXT,
  latitude REAL,
  longitude REAL
);

CREATE TABLE IF NOT EXISTS detections (
  detection_id INTEGER PRIMARY KEY,
  image_id INTEGER,
  class_id INTEGER,
  poly_wkt TEXT,
  x_min REAL,
  y_min REAL,
  x_max REAL,
  y_max REAL,
  confidence REAL,
  FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE 
);
"""

# is_accepted INTEGER NOT NULL DEFAULT 0,
# is_healthy INTEGER NOT NULL DEFAULT 0,
# is_damaged INTEGER NOT NULL DEFAULT 0,
# has_vcuts INTEGER NOT NULL DEFAULT 0,
# is_dead INTEGER NOT NULL DEFAULT 0,
# is_crowded INTEGER NOT NULL DEFAULT 0,
# is_occluded INTEGER NOT NULL DEFAULT 0,
# has_other_problem INTEGER NOT NULL DEFAULT 0,



# Functions

In [3]:
import gc
import torch

def delete_results_from_gpu_memory():
    """
    Explicitly manages memory after processing each image to prevent running out of GPU memory
    """
    global results_gpu
    del results_gpu 
    gc.collect() 
    torch.cuda.empty_cache() # Clears unoccupied cached memory

# Usage example:
    
# delete_results_from_gpu_memory()


In [4]:
import pandas as pd
import exif

def get_data_for_images_table(results_cpu) -> pd.DataFrame:
    """ 
    Saves data for a single image into an 'images' table in a SQLite database. 
    If the image contains embedded EXIF metadata, GIS coordinates are extracted and saved. and saves it in a SQLite database.
    If the database exists, one record is appended. Otherwise, a new database is created before adding the record.
    
    

    Args:
        image_path (str): Path to the image file.
        db_path (str): Path to the SQLite database file.
    Returns:
        None
    """

    image_height = results_cpu[0].orig_shape[0]
    image_width = results_cpu[0].orig_shape[1]

    with open(image_path, 'rb') as f:
        imgx = exif.Image(f)

    if imgx.has_exif:
        # to see all available exif_data use imgx.get_all()
        
        # timestamp
        timestamp = imgx.datetime
            
        # latitude
        d, m, s = imgx.gps_latitude
        latitude = d + m/60 + s/3600   
        if imgx.gps_latitude_ref == 'S':
            latitude = -latitude  

        # longitude
        d, m, s = imgx.gps_longitude
        longitude = d + m/60 + s/3600   
        if imgx.gps_longitude_ref == 'W':
            longitude = -longitude
        longitude
    else:
        timestamp = None
        latitude = None
        longitude= None
    
    return (image_width, image_height, timestamp, latitude, longitude)

# image_path = image_paths[0]
# results_gpu = rs.run_sam3_semantic_predictor(input_image_path=image_path, text_prompts=text_prompts)
# results_cpu = [r.cpu() for r in results_gpu] # copy results to CPU
# delete_results_from_gpu_memory() # Clear GPU memory after processing each image
# get_data_for_images_table(results_cpu)

In [5]:
import pandas as pd

def get_data_for_detections_table(results_cpu, image_id):

    # Process detection results (assuming one image for simplicity: results[0])
    result = results_cpu[0]

    # --- Extract Bounding Box Data into a DataFrame ---
    # The .boxes.data attribute is a tensor containing [x_min, y_min, x_max, y_max, confidence, class]
    boxes_data = result.boxes.data.tolist()
    df_boxes = pd.DataFrame(boxes_data, columns=['x_min', 'y_min', 'x_max', 'y_max', 'confidence', 'class_id'])

    # Add class names for readability
    # class_names = model.names
    # df_boxes['class_name'] = df_boxes['class_id'].apply(lambda x: class_names[int(x)])

    # --- Extract Segmentation Mask Data ---
    # Masks are more complex as they represent pixel-wise information or polygon points.
    # To put this into a DataFrame, you could store the polygon points list for each object.
    masks_data = []
    # Iterate over each detected object's mask
    for i, mask in enumerate(result.masks.xy):
        # mask.xy contains the polygon points as a list of [x, y] coordinates
        # You can associate this with the corresponding entry in the bounding box DataFrame

        poly_arr = mask
        poly_wkt = rs.conv_poly_from_array_to_wkt(poly_arr)

        masks_data.append({
            'image_path': image_path,
            'object_index': i, 
            'class_id': df_boxes.iloc[i]['class_id'], 
            'poly_wkt': poly_wkt})
        df_masks = pd.DataFrame(masks_data)  

    # merge df_masks and df_detections  
    df_detections = pd.merge(df_masks, df_boxes, how="outer", left_index=True, right_index=True)
    
    # clean database
    df_detections['image_id'] = image_id
    df_detections.rename(columns={'class_id_x': 'class_id'}, inplace=True)
    df_detections.drop(['image_path', 'object_index', 'class_id_y'], inplace=True, axis='columns')
    df_detections = df_detections.astype({'class_id': int, 'x_min': int, 'y_min': int, 'x_max': int, 'y_max': int})

    return df_detections

# populate_detections_table(results_cpu, image_id)    
    
    

    # # Connect to the SQLite database (creates if it doesn't exist)
    # conn = sqlite3.connect(db_path)
    # # cursor = conn.cursor()
    # df_detections.to_sql(name='detections', con=conn, if_exists='append', index=False)
    # conn.close()


# Main

In [6]:
# delete database
if os.path.exists(db_path):
    os.remove(db_path)

In [7]:
assert rs.check_gpu(), 'ERROR: GPU is unavailable.'

CUDA version: 12.8
GPU device name: NVIDIA GeForce RTX 3080 Laptop GPU


In [8]:
# connect to db. A new db is created if db_path does not exist.
con = sqlite3.connect(db_path)

# create new db if one does not exist; otherwise tables are not modified
con.executescript(schema_sql);

In [9]:
image_path = image_paths[0]
con.execute(f'SELECT COUNT(*) FROM images WHERE  image_path = "{image_path}"').fetchone()[0] > 0


False

In [10]:
for image_path in image_paths:
    
    # skip image if it is already in the database
    if con.execute(f'SELECT COUNT(*) FROM images WHERE  image_path = "{image_path}"').fetchone()[0] > 0:
        print(f'WARNING: Image {image_path} is already in the database. Skipping to next image.')
        continue
        
    # Detect objects in image
    results_gpu = rs.run_sam3_semantic_predictor(input_image_path=image_path, text_prompts=text_prompts)
    
    # Free up GPU memory in preparation for detecting objects in the next image
    # This is a work-around to prevent out-of-memory errors from the GPU
    # I move all results for further processing and use the GPU only for object detection.
    results_cpu = [r.cpu() for r in results_gpu] # copy results to CPU
    print('deleting results from GPU memory')       
    delete_results_from_gpu_memory() # Clear GPU memory after processing each image
    
    # populate 'images' table
    image_width, image_height, timestamp, latitude, longitude = get_data_for_images_table(results_cpu)
    sql = """
    INSERT INTO images (image_path, image_width, image_height, timestamp, latitude, longitude) 
    VALUES (?,?,?,?,?,?)
    RETURNING image_id
    """
    parameters = (image_path, image_width, image_height, timestamp, latitude, longitude,)
    ic(parameters)
    
    # sql = 'INSERT INTO images (image_path) VALUES (?) RETURNING image_id'
    try:
        image_id = con.execute(sql, parameters).fetchone()[0] # THE COMMA IN THE PARAMETERS TUPLE IS IMPORTANT
    except sqlite3.IntegrityError as e:
        print(f'ERROR: Image {image_path} already exists in {db_path}')
        raise e    
    con.commit()
    ic(image_id)
    
    # populate 'detections' table
    df_detections = get_data_for_detections_table(results_cpu, image_id)
    for i, r in df_detections.iterrows():
            # populate 'detections' table
        sql = ''' 
        INSERT INTO detections
            (image_id, class_id, poly_wkt, x_min, y_min, x_max, y_max, confidence)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?);
        '''
        parameters = (image_id, 0, r['poly_wkt'], r['x_min'], r['y_min'], r['x_max'], r['y_max'], r['confidence']) 
        con.execute(sql, parameters)
        con.commit()
    
    # print('building database') 
    # build_database(
    #     image_path=image_path,
    #     results=results_cpu,                             
    #     db_path=db_path
    # ) 
con.close()   
print('FINISHED')    

Ultralytics 8.4.9 🚀 Python-3.13.11 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 15992MiB)

image 1/1 /home/aubrey/Desktop/sam3-2026-01-31/20251129_152106.jpg: 1932x1932 25 coconut palm trees, 1230.5ms
Speed: 14.4ms preprocess, 1230.5ms inference, 5.7ms postprocess per image at shape (1, 3, 1932, 1932)
Results saved to /home/aubrey/Desktop/blog2026/runs/segment/predict145
deleting results from GPU memory


ic| parameters: ('20251129_152106.jpg',
                 1920,
                 1080,
                 '2025:11:29 15:21:06',
                 -17.73039628333333,
                 168.18940991666668)
ic| image_id: 1


Ultralytics 8.4.9 🚀 Python-3.13.11 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 15992MiB)

image 1/1 /home/aubrey/Desktop/sam3-2026-01-31/08hs-palms-03-zglw-superJumbo.webp: 1932x1932 2 coconut palm trees, 1134.4ms
Speed: 14.4ms preprocess, 1134.4ms inference, 1.0ms postprocess per image at shape (1, 3, 1932, 1932)
Results saved to /home/aubrey/Desktop/blog2026/runs/segment/predict146
deleting results from GPU memory
FINISHED


ic| parameters: ('08hs-palms-03-zglw-superJumbo.webp', 1366, 2048, None, None, None)
ic| image_id: 2


# Retrieve data from database

In [12]:
con = sqlite3.connect(db_path)
pd.read_sql('SELECT * FROM images', con)

,image_id,image_path,image_width,image_height,timestamp,latitude,longitude
0,1,20251129_152106.jpg,1920,1080,2025:11:29 15:21:06,-17.730396,168.18941
1,2,08hs-palms-03-zglw-superJumbo.webp,1366,2048,NaN,NaN,NaN


In [13]:
pd.read_sql('SELECT * FROM detections', con)

,detection_id,image_id,class_id,poly_wkt,x_min,y_min,x_max,y_max,confidence
0,1,1,0,"POLYGON ((1339 805, 1338 804, 1338 803, 1337 8...",1342.0,709.0,1462.0,992.0,0.649902
1,2,1,0,"POLYGON ((437 934, 437 935, 438 936, 439 936, ...",266.0,762.0,338.0,944.0,0.259521
2,3,1,0,"POLYGON ((538 795, 537 796, 535 796, 534 797, ...",491.0,794.0,561.0,961.0,0.698730
3,4,1,0,"POLYGON ((902 863, 902 865, 901 866, 901 867, ...",817.0,790.0,938.0,971.0,0.743164
4,5,1,0,"POLYGON ((906 838, 906 835, 906 838, 943 847, ...",938.0,801.0,963.0,990.0,0.479980
5,6,1,0,"POLYGON ((1681 715, 1680 716, 1680 717, 1675 7...",1592.0,715.0,1778.0,905.0,0.771484
6,7,1,0,"POLYGON ((553 936, 553 938, 554 939, 554 940, ...",557.0,801.0,682.0,958.0,0.348389
7,8,1,0,"POLYGON ((1130 942, 1130 945, 1129 946, 1129 9...",1046.0,801.0,1136.0,977.0,0.677734
8,9,1,0,"POLYGON ((1519 750, 1519 751, 1518 752, 1518 7...",1483.0,748.0,1624.0,902.0,0.616211
9,10,1,0,"POLYGON ((1013 829, 1008 834, 1006 834, 1003 8...",963.0,828.0,1055.0,985.0,0.763672


In [14]:
con.close()